## Introduction


Data provider [Quandl](https://www.quandl.com/) offers a vast array of free and paid databases, all accessible with the same Python API (application program interface). Quandl aggregates data from many sources, ranging from scientific to economic to government related topics. They conveniently provide the data in powerful Pandas DataFrames.

## Getting Data From Quandl


To use Quandl you will have to create an API key. The purpose of the API key is to make it easy for Quandl to track the usage of their data (creating data for them to study!) and for them to ensure that no one user is abusing their system with too many requests.

Create an API key by first creating an account on [Quandl](https://www.quandl.com/). You can log in with your Google, GitHub or LinkedIn accounts if you like.

After creating an account, access your *Account Settings* from the *Me* dropdown in the upper right corner. Then click on the *API KEY* link on the left below *PASSWORD*. Save the API Key: you'll need that in a moment. There is [documentation](https://www.quandl.com/docs/api?python#) available for the API. 

Quandl provides a Python module that allows for easy access to their API.  

In [2]:
import seaborn as sns
sns.set()
import pandas as pd
import quandl

Now, tell Quandl about the API key you created above:

In [3]:
quandl.ApiConfig.api_key = 'hp1krGfL8waVVz3Sxb5Q'  # Fill in your value here

## Bureau of Labor Statistics codes

We will use data provided by the US [Bureau of Labor Statistics](https://www.quandl.com/data/BLSE?keyword=). Among other things, they track monthly employment numbers by industry for each state.

We are specifically interested in their *State and Area Employment, Hours, and Earnings* data, as described in their [documentation](https://www.quandl.com/data/BLSE/documentation/documentation). The documentation describes the *Code Nomenclature* for data files for all of the combinations of states and industries and seasonally/not seasonally adjusted data.

Each of these data sets looks like this one:

https://www.quandl.com/data/BLSE/SMS01000004300000001-All-Employees-In-Thousands-Transportation-and-Utilities-Alabama

In this project, we will need to combine all of the data tables in this subgroup of the `BLS` database. There will be about 1118 tables in total. We will develop a process for downloading them one by one using `quandl.get()` on the Quandl codes for individual tables, and then concatenate them all into one data set.

Quandl provides a convenient database metadata file containing the `BLSE` data set codes and descriptions. We'll download this file and pick out the *State and Area Employment, Hours, and Earnings* data table codes based on the table descriptions.

In [11]:
!aws s3 sync s3://dataincubator-course/BLSE/BLSE_codes.zip .

Now, we can load all of the metadata into a dataframe:

In [4]:
blse_codes = pd.read_csv('BLSE-datasets-codes.csv', header=None,
                    names=('Code', 'Description'))
blse_codes.head()

,Code,Description
0,BLSE/SMU24000005051700001,"All Employees, In Thousands; Telecommunication..."
1,BLSE/SMU51000000700000001,"All Employees, In Thousands; Service-Providing..."
2,BLSE/BDU0000000000000000110004RQ5,Employment; Gross Job Losses; All industries; ...
3,BLSE/BDU0000000000000000110005RQ5,Employment; Contractions; All industries; Rate...
4,BLSE/CEU0000000001,"Employment - All employees, thousands; Total n..."


All of the tables with relevant employment information have a description that begins, "All Employees".  Create a new data frame that contains only those rows. 

In [5]:
valid_codes = blse_codes[blse_codes['Description'].str.startswith('All Employees')]

## Downloading data

We want to download and store the tables corresponding with each Quandl code in the `valid_codes` DataFrame.  Let's start by downloading the data from the beginning of 2006 to the end of 2015 corresponding with just one code:

In [35]:
code = valid_codes.loc[1, 'Code']
description = valid_codes.loc[1, 'Description']
print(code, description)

BLSE/SMU51000000700000001 All Employees, In Thousands; Service-Providing - Virginia


In [6]:
df = quandl.get('BLSE/SMU24000005051700001', start_date='2006-01-01', end_date='2015-12-31')

,Value
Date,
2006-01-31,20.2
2006-02-28,20.3
2006-03-31,20.4
2006-04-30,20.5
2006-05-31,20.6
...,...
2015-08-31,14.3
2015-09-30,14.2
2015-10-31,14.9


Now, we'll create a single function, so that we can reuse it for each `BLSE` data set.  The function should accept a Quandl code and table description, and use those arguments to retrieve the data set into a dataframe, adding columns for the state, category, and adjustment flag.

It's also a good idea to have this function write the dataframe to a file.  This way, if a data retrieval fails, we can rerun just that data set without needing to download all of the data again.

We can use Pandas' `to_pickle()` and `from_pickle()` functions, or another mechanism.  The checkpoint library [ediblepickle](https://pypi.python.org/pypi/ediblepickle/1.1.3) could also be used to streamline the process so that the time-consuming code will only be run when necessary.

In [8]:
import time
import regex as re
import quandl
import pandas as pd

quandl.ApiConfig.api_key = 'hp1krGfL8waVVz3Sxb5Q'

def get_data(code, description):
    # Download data
    # Add columns
    # Save locally
    # Return the dataframe
    time.sleep(0.1)
    df = quandl.get(code, start_date='2006-01-01', end_date='2015-12-31')
    parsing = [item.strip(' ') for item in (re.split(r'[,;-]\s+', description))]
    state = parsing[-1]
    category = parsing[-2]
    if code[7] == 'S':
        adjusted = True
    if code[7] == 'U':
        adjusted = False
   
    df['State'] = pd.Series(state, index=df.index)
    df['Category'] = pd.Series(category, index=df.index)
    df['Adjusted'] = pd.Series(adjusted, index=df.index)
    df.to_pickle('sample_df.pkl')
    return df


Test that function for several Quandl codes and table descriptions from `valid_codes` by changing `idx` in the cell below to make sure the function works as we expect.

In [9]:
idx = 0
code = valid_codes.iloc[idx]['Code']
description = valid_codes.iloc[idx]['Description']
test_df = get_data(code, description)
test_df

,Value,State,Category,Adjusted
Date,,,,
2006-01-31,20.2,Maryland,Telecommunications,False
2006-02-28,20.3,Maryland,Telecommunications,False
2006-03-31,20.4,Maryland,Telecommunications,False
2006-04-30,20.5,Maryland,Telecommunications,False
2006-05-31,20.6,Maryland,Telecommunications,False
...,...,...,...,...
2015-08-31,14.3,Maryland,Telecommunications,False
2015-09-30,14.2,Maryland,Telecommunications,False
2015-10-31,14.9,Maryland,Telecommunications,False


## Getting all data

Now that we are sure the function works correctly, we will iterate over `valid_codes`, passing each Quandl code and table description to the function.

Quandl limits the rate of API calls wecan make. To slow it down we can tell Python to `sleep` for a short time to keep it under the threshold.
```python
import time
time.sleep(0.1)  # sleep for 0.1 seconds (100 ms)
```

With that, we can load all of the data into a single dataframe using `pd.concat()` function and a comprehension to handle the iteration.

In [8]:
df_all = pd.concat(get_data(code, description) for code, description
                   in valid_codes.itertuples(index=False))
df_all.shape

We will split the data into two groups: adjusted or unadjusted data. Also, we will remove the `Total Private` and `Total Nonfarm` data, as these statistics are aggregations, not industries.

In [9]:
df_raw = df_all.query("Adjusted == False and ~Category.str.contains('Total Private|Total Nonfarm')")

df_raw.shape

(62160, 4)

In [10]:
df_adjusted = df_all.query("Adjusted == True and ~Category.str.contains('Total Private|Total Nonfarm')")
df_adjusted.shape

(46524, 4)

## Question 4: State/industry pairs

For this question, we will use the *unadjusted data* to find the 100 largest state-industry pairs for December 2015.

In [11]:
def df_selected(start_date, end_date):
    df_raw.index = pd.to_datetime(df_raw.index)
    mask = (df_raw.index >= start_date) & (df_raw.index <= end_date)
    df_selected = df_raw.loc[mask]
    return df_selected

In [12]:
dec15 = df_selected('2015-12-1', '2015-12-31')

In [13]:
# Sort them by 'Value' and choose the top 100
top100 = pd.DataFrame(dec15.groupby(['State', 'Category'])['Value'].sum().nlargest(100))

top100.reset_index(inplace=True)

In [14]:
state_industry_tuples = [((row.State, row.Category), (row.Value)*1000) for row in top100.itertuples(index=False)]
state_industry_tuples[0:5]

[(('California', 'Service-Providing'), 14362200.0),
 (('Texas', 'Service-Providing'), 10239200.0),
 (('New York', 'Service-Providing'), 8665800.0),
 (('Florida', 'Service-Providing'), 7551200.0),
 (('Illinois', 'Service-Providing'), 5244800.0)]

## Question 5: State total employed

Using the unadjusted data, we will calculate the total number of employed people in each state in December 2015.

In [15]:
state_total = pd.DataFrame(dec15.groupby('State')['Value'].sum())
state_total.reset_index(inplace=True)

state_total_employed = [(row.State, int(row.Value*1000)) for row in state_total.itertuples(index=False)]
state_total_employed[:3]

[('Alabama', 3008700), ('Alaska', 466400), ('Arizona', 3684400)]

## Question 6: State industry growth

Using the unadjusted data, we want to know the industry growth for each state from December 2006 to December 2015?

We want to compare rows in the `dec06` and `dec15` dataframes that have the same state and category.  When operations are conducted on dataframes, rows are matched by index.  Indices can have multiple levels.  Use the `.set_index()` method with a list as an argument to achieve this.

In [16]:
dec06 = df_selected('2006-12-1', '2006-12-31')
group06 = pd.DataFrame(dec06.groupby(['State', 'Category'])['Value'].sum())
group06.reset_index(inplace=True)

In [17]:
group15 = pd.DataFrame(dec15.groupby(['State', 'Category'])['Value'].sum())
group15.reset_index(inplace=True)

Now, we can do math directly on the dataframes.

In [18]:
df_growth = pd.merge(group06, group15, on=['State', 'Category'], suffixes=['_06', '_15'])
df_growth['growth'] = (df_growth['Value_15'] - df_growth['Value_06'])*100/df_growth['Value_06']

df_growth.head()

,State,Category,Value_06,Value_15,growth
0,Alabama,Durable Goods,188.0,170.0,-9.574468
1,Alabama,Goods Producing,422.7,354.4,-16.158032
2,Alabama,Government,380.2,384.1,1.025776
3,Alabama,Manufacturing,299.4,260.0,-13.159653
4,Alabama,Mining and Logging,13.1,10.2,-22.137405


In [60]:
# Assuming 'data' is the Series with multi-level indices
# Group the data by 'State' and find the index corresponding to the maximum growth in each group
max_growth_indices = df_growth.groupby('State')['growth'].idxmax()

# Create a list to store the tuples of states, industries, and percentages
state_industry_growth = []

# Iterate through each state and get the corresponding state, industry, and percentage values
for state, index in max_growth_indices.items():
    industry = df_growth['Category'].loc[index]
    percentage = df_growth['growth'].loc[index]
    state_industry_growth.append(((state, industry), percentage))

# 'largest_percent_growth' will be a list of tuples with 53 entries, each containing the state, industry, and percentage.
state_industry_growth[:3]

[(('Alabama', 'Transportation and Utilities'), 4.0221914008321855),
 (('Alaska', 'Mining and Logging'), 31.45161290322581),
 (('Arizona', 'Transportation and Utilities'), 17.142857142857142)]

We can pull out a group from our `DataFrameGroupBy` object for analysis.

In [ ]:
Write a function that takes this dataframe and returns the row with the maximum value.

In [19]:
def largest_value(df):
    return df.loc[df['growth'].idxmax()]

by_state = df_growth.groupby('State')
fastest_by_state = by_state.apply(lambda x:largest_value(x))
state_industry_growth = [((row.State, row.Category), row.growth) for row in fastest_by_state.itertuples(index=False)]
state_industry_growth[:3]

[(('Alabama', 'Transportation and Utilities'), 4.0221914008321855),
 (('Alaska', 'Mining and Logging'), 31.45161290322581),
 (('Arizona', 'Transportation and Utilities'), 17.142857142857142)]

## Question 7: Max employment

Using the unadjusted data, we want to find the number of people employed nationally in each industry during the month that industry peaked in our data set.

In [20]:
df_emp = pd.DataFrame(dec15.groupby('Category')['Value'].sum())
df_emp.reset_index(inplace=True)

In [21]:
max_employment = [(row.Category, row.Value) for row in df_emp.itertuples(index=False)]
max_employment

[('Air Transportation', 392.3),
 ('Commercial Banking', 101.5),
 ('Durable Goods', 7767.5),
 ('Goods Producing', 19786.5),
 ('Government', 23089.6),
 ('Manufacturing', 12455.9),
 ('Mining and Logging', 723.0),
 ('Rail Transportation', 15.200000000000001),
 ('Real Estate', 1050.0),
 ('Service-Providing', 126004.7),
 ('Telecommunications', 680.0),
 ('Transportation', 4.5),
 ('Transportation and Utilities', 5723.4),
 ('Truck Transportation', 1301.5),
 ('Water Transportation', 12.7),
 ('Wholesale Trade', 5800.1)]

## Question 8: Quarterly non-farm

Using the seasonally adjusted data, we want to calculate the quarterly percent change for national total non-farm employment. 

In [25]:
df_adjusted = df_all.query("Adjusted == True and Category.str.contains('Total Nonfarm')")
df_adjusted.reset_index(inplace=True)

In [30]:
df_quarterly = pd.DataFrame(df_adjusted.groupby('Date')['Value'].sum().resample('Q').last())
df_quarterly.reset_index(inplace=True)

In [31]:
df_quarterly['percentage_change'] = df_quarterly['Value'].pct_change()*100
df_quarterly.reset_index(inplace=True)
df_quarterly['Formated_Date'] = df_quarterly['Date'].dt.strftime('%Y-%m-%d')
df_quarterly[:3]

quarterly_nonfarm = [(row.Formated_Date, row.percentage_change) for row in df_quarterly[1:].itertuples()]
quarterly_nonfarm[:3]

[('2006-06-30', 0.33424149239995593),
 ('2006-09-30', 0.22400239945370437),
 ('2006-12-31', 0.3309307682082707)]

## Question 9: Third largest industry

Using the unadjusted data, what is the 3rd largest industry in each state in December 2015? We'll express the employment in the 3rd largest industry as a percentage of the state's total industry employment in December 2015.

In [32]:
state_sum = pd.DataFrame(group15.groupby('State')['Value'].sum())
state_sum[:3]

,Value
State,
Alabama,3008.7
Alaska,466.4
Arizona,3684.4


In [33]:
df_state = pd.merge(group15, state_sum, on=['State'], how='outer')
df_state['percentage'] = df_state['Value_x']*100/df_state['Value_y']
df_state

,State,Category,Value_x,Value_y,percentage
0,Alabama,Durable Goods,170.0,3008.7,5.650281
1,Alabama,Goods Producing,354.4,3008.7,11.779174
2,Alabama,Government,384.1,3008.7,12.766311
3,Alabama,Manufacturing,260.0,3008.7,8.641606
4,Alabama,Mining and Logging,10.2,3008.7,0.339017
...,...,...,...,...,...
513,Wyoming,Mining and Logging,21.7,428.7,5.061815
514,Wyoming,Service-Providing,238.4,428.7,55.609984
515,Wyoming,Transportation and Utilities,15.4,428.7,3.592256
516,Wyoming,Truck Transportation,4.2,428.7,0.979706


In [36]:
dec15_state = df_state.groupby('State')

In [206]:
large3 = dec15_state.get_group('Alabama').nlargest(3, 'percentage')
large3

,State,Category,Value_x,Value_y,percentage
5,Alabama,Service-Providing,1645.7,3008.7,54.698042
2,Alabama,Government,384.1,3008.7,12.766311
1,Alabama,Goods Producing,354.4,3008.7,11.779174


In [207]:
large3.tail(1)

,State,Category,Value_x,Value_y,percentage
1,Alabama,Goods Producing,354.4,3008.7,11.779174


In [34]:
def third_largest(df):
    return df.nlargest(3, 'percentage').tail(1)

In [37]:
AZ = dec15_state.get_group('Arizona')

In [38]:
third_largest(AZ)

,State,Category,Value_x,Value_y,percentage
22,Arizona,Goods Producing,303.6,3684.4,8.240148


In [39]:
df_third_largest = dec15_state.apply(lambda x:third_largest(x))
df_third_largest[:3]

,,State,Category,Value_x,Value_y,percentage
State,,,,,,
Alabama,1,Alabama,Goods Producing,354.4,3008.7,11.779174
Alaska,12,Alaska,Goods Producing,39.0,466.4,8.361921
Arizona,22,Arizona,Goods Producing,303.6,3684.4,8.240148


In [40]:
third_largest = [((row.State, row.Category), row.percentage) for row in df_third_largest.itertuples()]
third_largest[:3]

[(('Alabama', 'Goods Producing'), 11.779173729517732),
 (('Alaska', 'Goods Producing'), 8.361921097770155),
 (('Arizona', 'Goods Producing'), 8.240147649549453)]